In [1]:
import os
import sys
import boto3
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from dotenv import load_dotenv

# defining functions module path
module_path = str(Path.cwd().parents[0] / "src")
if module_path not in sys.path:
    sys.path.append(module_path)

from functions import connect_to_redshift

In [2]:
# removing warning messages
warnings.filterwarnings("ignore")

In [3]:
# Loading my .env file
load_dotenv()

True

In [4]:
# defining redshift credentials
dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [5]:
# establishing redshift connection
rs_connection = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [6]:
query = """select *
           from bootcamp.online_transactions_fixed
        """

In [7]:
ot_cleaned = pd.read_sql(query, rs_connection)

In [8]:
ot = ot_cleaned.copy()

In [9]:
ot_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399841 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            399841 non-null  object        
 1   stock_code         399841 non-null  object        
 2   description        399841 non-null  object        
 3   quantity           399841 non-null  int64         
 4   invoice_date       399841 non-null  datetime64[ns]
 5   price              399841 non-null  float64       
 6   customer_id        399841 non-null  object        
 7   country            399841 non-null  object        
 8   total_order_value  399841 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.5+ MB


In [10]:
ot_cleaned.describe()

,quantity,price,total_order_value
count,399841.000000,399841.000000,399841.000000
mean,12.224359,2.952914,20.716904
std,250.788860,7.317593,425.471765
min,-80995.000000,0.000000,-168469.600000
25%,2.000000,1.250000,4.250000
50%,5.000000,1.950000,11.580000
75%,12.000000,3.750000,19.500000
max,80995.000000,1599.260000,168469.600000


In [15]:
# Checking how many transactions are having price equal or less than 0
no_price_stock = ot_cleaned[ot_cleaned.price <= 0]['stock_code'].unique()
no_price = ot_cleaned[ot_cleaned.price <= 0]

print(f"There are {no_price.shape[0]} transactions with price zero. That represents {round(no_price.shape[0] * 100 / ot.shape[0], 2)} % of the total data")


There are 34 transactions with price zero. That represents 0.01 % of the total data


In [16]:
no_price.head()

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,total_order_value
12082,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,u12647,Germany,0.0
16519,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,u13081,United Kingdom,0.0
23249,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,u13081,United Kingdom,0.0
28960,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,u14911,EIRE,0.0
41276,550188,22636,CHILDS BREAKFAST SET CIRCUS PARADE,1,2011-04-14 18:57:00,0.0,u12457,Switzerland,0.0


In [17]:
no_price.describe(include=object)

,invoice,stock_code,description,customer_id,country
count,34,34,34,34,34
unique,28,34,34,25,8
top,564651,22841,ROUND CAKE TIN VINTAGE GREEN,u14646,United Kingdom
freq,4,1,1,4,21


In [18]:
print(f"There are {(ot_cleaned.quantity < 0).sum()} entries with negative quantity")

There are 8507 entries with negative quantity


In [19]:
ot_cleaned['invoice'][ot_cleaned.quantity < 0]

137       C536548
229       C536815
242       C536850
243       C536854
382       C536854
           ...   
399630    C581409
399664    C581148
399702    C581490
399759    C581316
399776    C581390
Name: invoice, Length: 8507, dtype: object

In [20]:
# Checking is all the invoice with negative quantity start with a letter C 
ot_cleaned['invoice'][ot_cleaned.quantity < 0].str.startswith('C').sum()

8507

### I will consider all the entries with negative quantity as returns. They have a C in the invoice code, which I will assume refers to a CANCELLED transaction

In [21]:
ot_cancelled = ot_cleaned[ot_cleaned.invoice.str.startswith('C')]

### I will clean my ot_cleaned by removing the transactions with negative quantity and the transactions with price equal to zero

In [22]:
# Dropping cancelled orders
ot_cleaned.drop(ot_cancelled.index, axis=0, inplace=True)

In [23]:
# Dropping transaction with no price
ot_cleaned.drop(no_price.index, axis=0, inplace=True)

In [24]:
ot_cleaned.describe(include=object)

,invoice,stock_code,description,customer_id,country
count,391300,391300,391300,391300,391300
unique,18405,3662,3627,4334,37
top,576339,85123A,CREAM HANGING HEART T-LIGHT HOLDER,u17841,United Kingdom
freq,542,2023,2023,7667,348919


In [25]:
ot_cleaned.country.sort_values().unique()

array(['Australia', 'Austria', 'Bahrain', 'Belgium', 'Brazil', 'Canada',
       'Channel Islands', 'Cyprus', 'Czech Republic', 'Denmark', 'EIRE',
       'European Community', 'Finland', 'France', 'Germany', 'Greece',
       'Iceland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'RSA',
       'Saudi Arabia', 'Singapore', 'Spain', 'Sweden', 'Switzerland',
       'USA', 'United Arab Emirates', 'United Kingdom', 'Unspecified'],
      dtype=object)

### There are transactions purchased in 37 different countries. One of them is "Unspecified"

### The number of unique descriptions is not compatible with the number of unique stock codes. Why?

In [32]:
# Getting the data grouped by description and unique stock_code
pd.DataFrame(ot_cleaned.groupby("description")['stock_code'].nunique().sort_values(ascending=False))

,stock_code
description,
UNKNOWN,18
"METAL SIGN,CUPCAKE SINGLE HOOK",3
ROUND BLUE CLOCK WITH SUCKER,2
SQUARE CHERRY BLOSSOM CABINET,2
RETRO PLASTIC 70'S TRAY,2
...,...
FOOT STOOL HOME SWEET HOME,1
FORKED CACTUS CANDLE,1
FOUR HOOK WHITE LOVEBIRDS,1


### It is interesting to see that there are some stock codes with 'Unknown' description. 

In [33]:
# Filtering the data to get the number of stock codes with 'Unknown' description
stock_code_unknown = ot_cleaned[ot_cleaned.description == 'UNKNOWN']['stock_code'].unique()
print(f"There are {stock_code_unknown.shape[0]} stock codes with unknown description")

There are 18 stock codes with unknown description


In [34]:
# Checking if the stock codes with unknown description were sold in all countries
ot_cleaned[ot_cleaned['stock_code'].isin(stock_code_unknown)].country.unique()

array(['United Kingdom', 'Belgium', 'Switzerland', 'Poland',
       'Unspecified', 'Germany', 'EIRE', 'France', 'Norway', 'Spain',
       'Singapore', 'Sweden', 'Canada', 'Austria', 'Italy'], dtype=object)

### The Unknown description purchases were made across 15 of the 37 countries. The reason why they are Unknown is still unclear

### Concerning the descriptions with more than one stock code. Based on the cell below I will infer that they are the same product on different presentations (i.e size)

In [35]:
# Checking an example of an product with two stock_codes and just oe description 
ot_cleaned.groupby(["description"]).get_group("COLOURING PENCILS BROWN TUBE").head()

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,total_order_value
4098,536621,10135,COLOURING PENCILS BROWN TUBE,10,2010-12-02 10:35:00,1.25,u13108,United Kingdom,12.50
5523,537129,10135,COLOURING PENCILS BROWN TUBE,10,2010-12-05 12:15:00,1.25,u13506,United Kingdom,12.50
7340,536446,10133,COLOURING PENCILS BROWN TUBE,5,2010-12-01 12:15:00,0.85,u15983,United Kingdom,4.25
8244,537155,10133,COLOURING PENCILS BROWN TUBE,3,2010-12-05 13:05:00,0.85,u12748,United Kingdom,2.55
9707,537126,10135,COLOURING PENCILS BROWN TUBE,1,2010-12-05 12:13:00,1.25,u18118,United Kingdom,1.25


### Let's explore each column

In [ ]:
print(f"The data contains information about 391334 ({ot_cleaned.invoice.nunique()} invoices in total) online purchases made from {ot_cleaned.invoice_date.min()} to {ot_cleaned.invoice_date.max()}. {ot_cancelled.shape[0]} are cancelled orders")

In [ ]:
print(f"A total of {ot_cleaned.stock_code.nunique()} stock codes were sold from which {ot_cancelled.stock_code.nunique()} stocks were return at least once")

In [ ]:
inv_country = ot_cleaned.groupby('country').count()['invoice'].sort_values(ascending=False)

In [ ]:
print(f"The invoices were purchased in 37 countries being {inv_country.index[0]} the country with the most purchases ({inv_country[0]}) and {inv_country.index[-1]} with the least ({inv_country[-1]})")

In [ ]:
print(f"In total, there are {ot_cleaned.customer_id.nunique()} costumers")

### In the table below are shown the top 10 stock codes sold, their description, amount sold, their price and in how many country they had been sold

In [ ]:
# Let's get top 5 stock codes on the table 
# defining aggregation functions to show on the table
agg_dict = {'description': 'first', 'quantity': 'sum', 'price': 'first', 'country': 'nunique'}
top10 = ot_cleaned.groupby("stock_code").agg(agg_dict).sort_values(by='quantity', ascending=False)[:10]
top10

In [ ]:
# What are the countries with the highest purchase
ot.groupby('country').sum().sort_values(by='total_order_value', ascending=False)[:10]

### Let's explore now the data with returns

In [ ]:
ot_cancelled.groupby("stock_code").agg(agg_dict).sort_values(by='quantity')[:10]

In [ ]:
# Checking how many stock codes from the top10 best sellers are in the returned list
canc_grouped = ot_cancelled.groupby('stock_code').sum()['quantity'].sort_values()[:10]
top10.index.isin(canc_grouped.index).sum()

### Inspecting the returned items, I realized that in the top 10 most returned items are some of the best selling items.

In [ ]:
# Returns grouped by customer
ot_cancelled.groupby("customer_id").agg(agg_dict)

### 1534 customers returned a product at least once

In [ ]:
# Returns grouped by stock code
ot_cancelled.groupby("stock_code").agg(agg_dict)

### 1916 have been returned at least once

In [ ]:
ot_cancelled.describe(include=object)

### The most returned item is "REGENCY CAKESTAND 3 TIER" which has been returned 180 times. The country with most returns is United Kingdom with 7218 returns

### Let's explore the numerical values on the dataset

In [ ]:
# Let's start by exploring the outliers in the price column
_, mean_price, std_price, min_price, Q1_price, Q2_price, Q3_price, max_price = ot.price.describe()
IQR_price = Q3_price - Q1_price
low_lim_price = Q1_price - 1.5 * IQR_price
up_lim_price = Q3_price + 1.5 * IQR_price

In [ ]:
# Price histogram plotting
plt.hist(ot.price[ot['price'] < up_lim_price])
plt.axvline(x=Q2_price, color='red', label='median')
plt.axvline(x=up_lim_price, color='green', label='outlier upper limit')
plt.title("Price distribution without outliers")
plt.legend(loc='upper right')
plt.tight_layout()
plt.xlabel('Price (pounds)');

In [ ]:
outlier_perc = round(ot[ot['price'] >= up_lim_price].shape[0] * 100 / ot.shape[0], 2)
print(f"{outlier_perc} % of my data are outliers on the price variable")